In [1]:
import sys
import numpy as np
import cv2
import sklearn.metrics as sklm

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

# from keras import backend as K
# img_dim_ordering = 'tf'
# K.set_image_dim_ordering(img_dim_ordering)


### 1.데이터 불러오기

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 5s 0us/step


### 2. 데이터 사이즈 조정

In [3]:
# converting it to RGB
x_train = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_train]
x_train = np.concatenate([arr[np.newaxis] for arr in x_train]).astype('float32')

In [4]:
x_test = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_test]
x_test = np.concatenate([arr[np.newaxis] for arr in x_test]).astype('float32')

In [11]:
x_train[1][1]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [12]:
from tensorflow.keras.utils import to_categorical

In [13]:
y_train_one = to_categorical(y_train)
y_test_one = to_categorical(y_test)

### 3. 모델생성

In [14]:
img_shape = x_train.shape[1:]
img_shape

(32, 32, 3)

In [15]:
num_classes = len(set(y_train))
num_classes

10

In [16]:
layer_type="relu"
layer_type

'relu'

In [18]:
test_model = VGG16(weights='imagenet', include_top=False)

In [19]:
keras_input = Input(shape=img_shape, name = 'custom_input')

In [20]:
output_vgg16_conv = test_model(keras_input)

In [17]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
#model_vgg16_conv.summary()

# top layer 수정
keras_input = Input(shape=img_shape, name = 'custom_input')

# 커스탐모델 생성 
output_vgg16_conv = model_vgg16_conv(keras_input)
output_vgg16_conv

58892288/58889256 [==============================] - 43s 1us/step


<tf.Tensor 'vgg16/Identity:0' shape=(None, 1, 1, 512) dtype=float32>

In [21]:
# Fully Connected Layer 추가 생성 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(64, activation="relu", name='custom_1')(x)
x = Dense(64, activation="relu", name='custom_2')(x)
x = Dense(num_classes, activation='softmax', name='prediction')(x)
# 커스텀 모델 생성
pretrained_model = Model(inputs=keras_input, outputs=x)

In [22]:
pretrained_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
custom_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
custom_1 (Dense)             (None, 64)                32832     
_________________________________________________________________
custom_2 (Dense)             (None, 64)                4160      
_________________________________________________________________
prediction (Dense)           (None, 10)                650       
Total params: 14,752,330
Trainable params: 14,752,330
Non-trainable params: 0
_________________________________________________

### 4. 모델 컴파일

In [23]:
# Sparse_categorical_crossentropy : one hot encoding 없이도 학습이 가능함
pretrained_model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

### 5. 모델 훈련

In [24]:
pretrained_model.fit(x_train, 
                     y_train_one, 
                     epochs = 2, 
                     validation_data=(x_test, y_test_one), verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
41024/60000 [===================>..........] - ETA: 12:21 - loss: 0.6744 - accuracy: 0.7810

KeyboardInterrupt: 